# JupyterNotebookによる運用（辛口）

依頼：

admin/adminでログインできるはずのWordPressが入ったEC2のマシンイメージあるが、起動してもブラウザで動作を確認できない。調査して復旧させて欲しい。

OSはCentOS7で、WordPressはwp-cliというツールでメンテナンスできる状態とのこと。

参考情報：

[システムサービスの管理](https://access.redhat.com/documentation/ja-jp/red_hat_enterprise_linux/7/html/system_administrators_guide/sect-managing_services_with_systemd-services)

[WP-CLI Commands](https://developer.wordpress.org/cli/commands/config/)

まずは、起動してみましょう。

## ユーザ識別のため、ユーザ名を設定

In [ ]:
username = 'user99@nb12.ecloud.nii.ac.jp'

## awscliの準備

### インストール

In [ ]:
!sudo apt-get update && sudo apt-get upgrade -y && sudo apt-get install -y awscli

In [ ]:
!aws --version

### 設定

In [ ]:
target_region = 'ap-northeast-1'
!aws configure set default.region {target_region}
!aws configure set default.output json

認証情報としてAccessKeyIdとSecretAccessKeyを設定します。

コマンドに対する対話的な操作が必要ですから、JupyterのTerminal(treeページの[New] - [Terminal]から選択できます)から、 `aws configure` を実施してください。

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [ap-northeast-1]: (Enter)
Default output format [json]: (Enter)
```

### boto3ライブラリの追加

ansibleからawsを触るために、ライブラリを追加しておきます。

In [ ]:
!conda install -y boto3 boto

## keypairの作成

クラウド上でLinuxマシンを起動する前に、ssh接続に必要となるkeypairを作成します。

In [ ]:
!aws ec2 delete-key-pair --key-name {username}
row = !aws ec2 create-key-pair --key-name {username} --query 'KeyMaterial' --output text

with open(username + '.pem', mode='w') as f:
    f.write('\n'.join(row))

import os
os.chmod(username + '.pem', 0o600)

#!aws ec2 delete-key-pair --key-name {username}

## Linuxマシンを起動する

ansibleでec2インスタンスを起動します。ディストリビューションはUbuntuとします。

In [ ]:
image = 'ami-0b6f624ac10bbda43' # wp
instance_type = 't3.medium'
subnet = 'subnet-8a6618d1' # 先に作成済みのもの
sg = 'seminar-lc4ri' # 先に作成済みのもの

import json

row = !ansible localhost -m ec2 -a "region={target_region} key_name={username} \
profile=default wait=yes assign_public_ip=yes \
image={image} instance_type={instance_type} vpc_subnet_id={subnet} group={sg}"

while row[0].find('{') == -1:
    del row[0]
row[0] = "{"
linux = json.loads(''.join(row))
print(linux)

起動が完了するまで待ちます。

In [ ]:
!aws ec2 wait instance-status-ok --include-all-instances --instance-ids {linux['instance_ids'][0]}

起動したLinuxマシンでOSアップデートを実行します。

まず、接続情報を作成します。

In [ ]:
host = linux["instances"][0]["public_ip"]
ansible_user = "centos"
ansible_ssh_private_key_file = username + ".pem"

with open('hosts', mode='w') as f:
    f.write(host + ' ansible_user=' + ansible_user + ' ansible_ssh_private_key_file='
            + ansible_ssh_private_key_file)

作成した接続情報を使う前に、対象となるLinuxマシンを信頼できるホストとして、fingerprint登録しておきます。JupyterのTerminalを使って、ダミーのssh実行を行うと登録されます。

```
$ ssh a@（対象となるLinuxマシンのIPアドレス）
（中略）
Are you sure you want to continue connecting (yes/no)? yes←信頼しますよの意味で入力する
（以下略）
```

ssh接続を成功させたいわけではないので、接続自体は失敗させてしまって構いません。

実務では、ansibleの対象となるLinuxマシンが大量だったり不定だったりするかも知れません。その場合は、sshやansibleの設定で、fingerprint登録の有無を確認しないこともできます。

ansibleでコマンドを実行してLinuxマシンの起動を確認します。

In [ ]:
!ansible -i hosts all -a "pwd"

## 運用作業

依頼（再掲）：

admin/adminでログインできるはずのWordPressが入ったEC2のマシンイメージあるが、起動してもブラウザで動作を確認できない。調査して復旧させて欲しい。

OSはCentOS7で、WordPressはwp-cliというツールでメンテナンスできる状態とのこと。

参考情報：

[システムサービスの管理](https://access.redhat.com/documentation/ja-jp/red_hat_enterprise_linux/7/html/system_administrators_guide/sect-managing_services_with_systemd-services)

[WP-CLI Commands](https://developer.wordpress.org/cli/commands/config/)

写真付きの投稿ができるように復旧させてください。


## Linuxマシンを破棄する

In [ ]:
!aws ec2 terminate-instances --instance-ids {linux['instance_ids'][0]}